In [ ]:
# Environment variables for jwst pipeline
import os


# Set the environment variables
os.environ['CRDS_PATH'] = os.path.expanduser('~/crds_cache')
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'


for k in ['CRDS_SERVER_URL','CRDS_PATH']:
    if k not in os.environ:
        print(f"Environment variable {k} not set, `jwst` pipeline won't work")
    else:
        print(f"{k}: {os.environ[k]}")
import os
import glob
import yaml
import warnings

import numpy as np
import matplotlib.pyplot as plt

import grizli
from grizli import utils, jwst_utils
jwst_utils.set_quiet_logging()
utils.set_warnings()

import astropy.io.fits as pyfits
from jwst.datamodels import SlitModel
import jwst

import msaexp
from msaexp import pipeline

print(f'grizli version = {grizli.__version__}')
print(f'msaexp version = {msaexp.__version__}')
print(f'jwst version = {jwst.__version__}')  